In [35]:
# imports
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as BS
import time
from lxml import etree
import sqlite3
import sqlalchemy

## Parse RussianFood.com

In [2]:
def scrape_recipes_title_links(page_source):
    recipe_list = []
    html = BS(page_source.content, 'html.parser')
    
    items = html.select('.recipe_list_new > .recipe_l > .title_o >.title')
    if(len(items)):
        for el in items:
            title = el.select('a')
            recipe_list.append((title[0].text, title[0]['href']))
    return recipe_list

In [56]:
# Парсинг названия рецепта со страницы
def scrape_recipe_title(page_source):
    html = BS(page_source.content, 'html.parser')
    title = html.select('h1')[0].text
    return title

In [88]:
# Парсинг описания рецепта со страницы
def scrape_recipe_desctiption(page_source):
    html = BS(page_source.content, 'html.parser')
    description = html.find('td', class_ = 'padding_l padding_r').find('p').text
    return description  

In [156]:
# Парсинг ссылки на фото рецепта со страницы
def scrape_recipe_photo(page_source):
    html = BS(page_source.content, 'html.parser')
    photo = html.find('table', class_='main_image').find('img')['src']
    photo=photo.replace('sm', 'big')
    return photo

In [182]:
# Парсинг ингредиентов со страницы
def scrape_recipe_ingredients(page_source):
    html = BS(page_source.content, 'html.parser')
    ingredients = html.find('table', class_='ingr').find_all('span')
    ingredients = [ingredient.text for ingredient in ingredients]
    return ingredients

In [186]:
# Парсинг рецепта целиком
def scrape_recipe(page_source):
    
    recipe_dict = {}
    
    # recipe title
    recipe_title = scrape_recipe_title(page_source)
    recipe_dict['title'] = recipe_title

    # recipe description 
    recipe_description =  scrape_recipe_desctiption(page_source)
    recipe_dict['description'] = recipe_description
    
    
   # recipe photo 
    recipe_photo = scrape_recipe_photo(page_source)
    recipe_dict['photo'] = recipe_photo

    # recipe ingredients
    recipe_ingredients = scrape_recipe_ingredients(page_source)
    recipe_dict['ingredients'] = recipe_ingredients

    return recipe_dict

In [201]:
# Парсинг страницы с рецептами
def scrape_page(recipe_links): 
    recipe_subcat_dict = {}
    for title,link in recipe_links:
        r = requests.get('https://www.russianfood.com'+link)
        recipe_dict = scrape_recipe(r)
        print(recipe_dict)
        recipe_subcat_dict[link] = recipe_dict
        time.sleep(45)
    return recipe_subcat_dict

In [194]:
page_source = 'https://www.russianfood.com/recipes/bytype/?fid=3#rcp_list'
r = requests.get(page_source)
recipes_1 = scrape_recipes_title_links(r)

In [195]:
recipes_1

[('Жаркое по-деревенски', '/recipes/recipe.php?rid=159506'),
 ('Куриный беф-строганов', '/recipes/recipe.php?rid=121844'),
 ('Картошка, запечённая с сосисками, помидорами и сыром',
  '/recipes/recipe.php?rid=165007'),
 ('Макароны с мясом в томатном соусе, на сковороде',
  '/recipes/recipe.php?rid=164739'),
 ('Жюльен с курицей и грибами', '/recipes/recipe.php?rid=140728'),
 ('Ленивые голубцы в духовке', '/recipes/recipe.php?rid=150532'),
 ('Запеканка мясная "Сытная"', '/recipes/recipe.php?rid=125466'),
 ('Тушёная картошка с мясом, грибами и сметаной',
  '/recipes/recipe.php?rid=165787'),
 ('Спаггети с курицей и помидорами черри', '/recipes/recipe.php?rid=145987'),
 ('Джамбалайя', '/recipes/recipe.php?rid=152930'),
 ('Гречка по-купечески', '/recipes/recipe.php?rid=134577'),
 ('Курица, тушенная в кисло-сладком соусе с болгарским перцем и морковью',
  '/recipes/recipe.php?rid=157262'),
 ('Фаршированный картофель, запечённый в духовке',
  '/recipes/recipe.php?rid=149801'),
 ('Лагман с куриц

In [203]:
start_link = "https://www.russianfood.com/recipes/bytype/?fid=3"
r = requests.get(start_link)

# Получаем ссылки на рецепты со страницы
recipe_links = scrape_recipes_title_links(r)

final_dict=scrape_page(recipe_links)
final_dict

{'title': 'Жаркое по-деревенски', 'description': 'Простое, но очень ароматное и вкусное блюдо для семейного обеда или ужина.', 'photo': '//img1.russianfood.com/dycontent/images_upl/465/big_464858.jpg', 'ingredients': ['Продукты ', 'Свиная корейка без кости - 1 кг', 'Сало свиное свежее - 50 г', 'Масло сливочное - 50 г', 'Картофель - 1 кг', 'Перец болгарский - 1 шт.', 'Морковь - 2 шт.', 'Лук репчатый - 2 шт.', 'Чеснок - 2 зубчика', 'Зелень петрушки свежая - по вкусу', 'Зелень петрушки сушёная - по вкусу', 'Специи для мяса - по вкусу', 'Перец чёрный горошком - по вкусу', 'Перец чёрный молотый - по вкусу', 'Соль - по вкусу']}
{'title': 'Куриный беф-строганов', 'description': 'Бефстроганов из курицы - очень нежный и вкусный. И быстро готовится. Готовится так же, как и бефстроганов из говядины, но меньше по времени.', 'photo': '//img1.russianfood.com/dycontent/images_upl/32/big_31172.jpg', 'ingredients': ['Продукты ', '(на 4 порции)', 'Куриное филе – 600 г', 'Сметана – 2 ст. ложки', 'Томатна

{'title': 'Фаршированный картофель, запечённый в духовке', 'description': 'Картошка в духовке, запёченная по этому рецепту, подойдёт как основное блюдо и как гарнир. Готовится очень просто, а получается сытный и ароматный картофель в духовке, выглядит аппетитно, а пахнет просто умопомрачительно!', 'photo': '//img1.russianfood.com/dycontent/images_upl/311/big_310322.jpg', 'ingredients': ['Продукты ', '(на 4 порции)', 'Картофель большой - 4 шт.', 'Бекон (или грудинка) - 120 г', 'Лук репчатый - 1 шт.', 'Сыр твёрдый - 70 г', 'Масло сливочное - 20 г', 'Соль - по вкусу', 'Масло растительное (при необходимости) - для жарки']}
{'title': 'Лагман с курицей', 'description': 'Лагман является традиционным национальным блюдом среднеазиатских народностей. Готовится блюдо из мяса (баранины) с овощами и лапшой. Я решила приготовить лагман на свой лад - из того, что было. Итак, готовим лагман из курицы.', 'photo': '//img1.russianfood.com/dycontent/images_upl/130/big_129813.jpg', 'ingredients': ['Продукт

{'title': 'Лаваш с сыром, колбасой, помидорами и яйцами', 'description': 'Иногда мы голову ломаем, что быстро приготовить на завтрак. Этот рецепт выручит в таких ситуациях. Очень вкусный и быстрый завтрак за 15-20 минут вместе с приготовлением. В результате получается настоящая вкуснятина из лаваша с сыром, колбасой, помидорами и яйцами.', 'photo': '//img1.russianfood.com/dycontent/images_upl/328/big_327983.jpg', 'ingredients': ['Продукты ', 'Лаваш тонкий - 1 большой лист', 'Сыр - 100 г', 'Колбаса - 100 г', 'Помидор (или соленый огурец) - около 60 г', 'Яйца - 2 шт.', 'Соль - по вкусу', 'Перец чёрный молотый - по вкусу', 'Кунжут (по желанию) - по вкусу']}
{'title': 'Картошка, тушенная с курицей в сметанно-сливочном соусе', 'description': 'Куриное филе и картофель в соусе из сметаны и сливок покорят с первого кусочка!) Курица и картошка в нежном соусе с добавлением тимьяна приобретают изумительный аромат, замечательный вкус и сочность. А ещё это блюдо готовится достаточно просто, и всего

{'title': 'Картофель с курицей и помидорами, запечённые в пергаменте', 'description': 'Быстро, вкусно, порционно, а главное - оригинальная подача. Блюдо запекается вот в таких интересных лодочках из пергамента. Каждая лодочка - одна порция. Картофель, лук, помидоры и сыр выкладываются в пергаментную лодочку и отправляются в духовку. Это вкусно, сытно и удобно!', 'photo': '//img1.russianfood.com/dycontent/images_upl/586/big_585155.jpg', 'ingredients': ['Продукты ', '(на 4 порции)', 'Картофель - 270 г', 'Куриное филе - 350 г', 'Помидор - 150 г', 'Лук репчатый - 100 г', 'Чеснок - 2-3 зубчика', 'Сыр твёрдый - 100 г', 'Карри - 0,5 ч. ложки', 'Соль - по вкусу', 'Перец чёрный молотый - по вкусу', 'Петрушка свежая (для подачи) - 1-2 веточки']}
{'title': 'Запеканка "Ленивая жена"', 'description': 'Рецепт из серии "проще простого" или "как удивить любимых без особого труда".', 'photo': '//img1.russianfood.com/dycontent/images_upl/33/big_32659.jpg', 'ingredients': ['Продукты ', '(на 4 порции)', '

{'title': 'Рыба, тушенная с болгарским перцем в томатном соусе', 'description': 'Тилапия, тушенная с овощами в томатном соусе, готовится просто и быстро, но при этом радует своей сочностью и ароматом. В отличие от классических рецептов, помимо моркови и лука также используем болгарский перец и помидоры. А чтобы вкус был ещё интереснее, добавим соевый соус и уксус.', 'photo': '//img1.russianfood.com/dycontent/images_upl/612/big_611037.jpg', 'ingredients': ['Продукты ', '(на 6 порций)', 'Тилапия (филе) - 700 г (6 шт.)', 'Перец болгарский - 200 г (2 шт.)', 'Помидоры - 300 г (2 шт.)', 'Морковь (некрупная) - 150 г (2 шт.)', 'Лук репчатый - 140 г (1 шт.)', 'Чеснок - 3 зубчика', 'Томатный соус - 100 г', 'Соевый соус - 1 ст. ложка', 'Уксус столовый - 1 ст. ложка', 'Масло растительное - 30-45 мл (2-3 ст. ложки)', 'Паприка молотая - 1 ч. ложка', 'Лавровый лист - 1 шт.', 'Перец красный острый молотый - щепотка', 'Перец чёрный молотый - 2 щепотки', 'Соль - по вкусу']}
{'title': 'Тушёная тыква с ко

{'title': 'Кефтедес - котлеты по-гречески', 'description': 'Греческие котлеты (вообще-то даже кипрские) "кефтедес" отличаются от обычных тем, что в состав котлетного теста входит большое количество овощей и пряных трав. Поэтому котлеты по-кипрски получаются очень сочные и ароматные.', 'photo': '//img1.russianfood.com/dycontent/images_upl/39/big_38168.jpg', 'ingredients': ['Продукты ', 'Фарш мясной - 500 г', 'Яйцо - 1 шт.', 'Хлеб черствый - 3-4 куска', 'Молоко - 100 мл', 'Яблоки - 100 г', 'Кабачок - 100 г', 'Лук - 1 шт.', 'Зелень петрушки - 0,5 пучка', 'Зелень кинзы - 0,5 пучка', 'Базилик - 3 веточки', 'Соль - 1 ч. ложка', 'Перец черный молотый - 0,25 ч. ложки', 'Масло для жарки - 100-200 мл']}
{'title': 'Отбивные из свинины в яблочном кляре', 'description': 'Замаринуйте свиные отбивные с натёртыми яблоками, а после обжарьте их в кляре. Кислинка яблок размягчает мясо, свинина приобретает фруктовый аромат и кисло-сладкий вкус. Отбивные получаются нежными и сочными, с неповторимым вкусом.

{'/recipes/recipe.php?rid=159506': {'title': 'Жаркое по-деревенски',
  'description': 'Простое, но очень ароматное и вкусное блюдо для семейного обеда или ужина.',
  'photo': '//img1.russianfood.com/dycontent/images_upl/465/big_464858.jpg',
  'ingredients': ['Продукты ',
   'Свиная корейка без кости - 1 кг',
   'Сало свиное свежее - 50 г',
   'Масло сливочное - 50 г',
   'Картофель - 1 кг',
   'Перец болгарский - 1 шт.',
   'Морковь - 2 шт.',
   'Лук репчатый - 2 шт.',
   'Чеснок - 2 зубчика',
   'Зелень петрушки свежая - по вкусу',
   'Зелень петрушки сушёная - по вкусу',
   'Специи для мяса - по вкусу',
   'Перец чёрный горошком - по вкусу',
   'Перец чёрный молотый - по вкусу',
   'Соль - по вкусу']},
 '/recipes/recipe.php?rid=121844': {'title': 'Куриный беф-строганов',
  'description': 'Бефстроганов из курицы - очень нежный и вкусный. И быстро готовится. Готовится так же, как и бефстроганов из говядины, но меньше по времени.',
  'photo': '//img1.russianfood.com/dycontent/images_upl

In [297]:
df = pd.DataFrame(final_dict)
df=df.T
df.reset_index(inplace=True)
df.rename(columns={'index':'link'},inplace = True)
df=df[['title', 'link', 'description','ingredients', 'photo']]
for x in range(len(df['ingredients'])):
    df['ingredients'][x]=",".join([str(i) for i in df['ingredients'][x]])

In [233]:
df.to_csv('parsed_rec.csv', encoding='UTF-16')

In [333]:
df.head()

,title,link,description,ingredients,photo,combined_text
0,Жаркое по-деревенски,/recipes/recipe.php?rid=159506,"Простое, но очень ароматное и вкусное блюдо дл...","Продукты ,Свиная корейка без кости - 1 кг,Сало...",//img1.russianfood.com/dycontent/images_upl/46...,"Жаркое по-деревенскиПростое, но очень ароматно..."
1,Куриный беф-строганов,/recipes/recipe.php?rid=121844,Бефстроганов из курицы - очень нежный и вкусны...,"Продукты ,(на 4 порции),Куриное филе – 600 г,С...",//img1.russianfood.com/dycontent/images_upl/32...,Куриный беф-строгановБефстроганов из курицы - ...
2,"Картошка, запечённая с сосисками, помидорами и...",/recipes/recipe.php?rid=165007,"Простое, но достаточно интересное блюдо - запе...","Продукты ,(на 4 порции),Картофель (некрупный) ...",//img1.russianfood.com/dycontent/images_upl/55...,"Картошка, запечённая с сосисками, помидорами и..."
3,"Макароны с мясом в томатном соусе, на сковороде",/recipes/recipe.php?rid=164739,"Блюдо привлекательно тем, что макароны отдельн...","Продукты ,Свинина (корейка без кости) - 300 г,...",//img1.russianfood.com/dycontent/images_upl/55...,"Макароны с мясом в томатном соусе, на сковород..."
4,Жюльен с курицей и грибами,/recipes/recipe.php?rid=140728,Очень популярную закуску - жульен (или точнее ...,"Продукты ,(на 4 порции),Филе куриное - 400 г,Г...",//img1.russianfood.com/dycontent/images_upl/19...,Жюльен с курицей и грибамиОчень популярную зак...


In [303]:
engine = sqlalchemy.create_engine('sqlite:///My_DB.db')
df.to_sql('recipes', engine, if_exists='append', index=False)

In [307]:
pd.read_sql('SELECT * FROM recipes', engine)

,id,title,link,description,ingredients,photo
0,1,Жаркое по-деревенски,/recipes/recipe.php?rid=159506,"Простое, но очень ароматное и вкусное блюдо дл...","Продукты ,Свиная корейка без кости - 1 кг,Сало...",//img1.russianfood.com/dycontent/images_upl/46...
1,2,Куриный беф-строганов,/recipes/recipe.php?rid=121844,Бефстроганов из курицы - очень нежный и вкусны...,"Продукты ,(на 4 порции),Куриное филе – 600 г,С...",//img1.russianfood.com/dycontent/images_upl/32...
2,3,"Картошка, запечённая с сосисками, помидорами и...",/recipes/recipe.php?rid=165007,"Простое, но достаточно интересное блюдо - запе...","Продукты ,(на 4 порции),Картофель (некрупный) ...",//img1.russianfood.com/dycontent/images_upl/55...
3,4,"Макароны с мясом в томатном соусе, на сковороде",/recipes/recipe.php?rid=164739,"Блюдо привлекательно тем, что макароны отдельн...","Продукты ,Свинина (корейка без кости) - 300 г,...",//img1.russianfood.com/dycontent/images_upl/55...
4,5,Жюльен с курицей и грибами,/recipes/recipe.php?rid=140728,Очень популярную закуску - жульен (или точнее ...,"Продукты ,(на 4 порции),Филе куриное - 400 г,Г...",//img1.russianfood.com/dycontent/images_upl/19...
...,...,...,...,...,...,...
60,61,"Мясо по-немецки с белым вином, помидорами и яб...",/recipes/recipe.php?rid=124762,"Изысканное мясо - свинина, тушенная в духовке ...","Продукты ,(на 5 порций),Свинина (мякоть) - 750...",//img1.russianfood.com/dycontent/images_upl/52...
61,62,"Куриные бёдрышки, запечённые с яблоками в медо...",/recipes/recipe.php?rid=167925,"Куриные бёдрышки с яблоками, запечённые с мёдо...","Продукты ,(на 3 порции),Куриные бёдрышки – 850...",//img1.russianfood.com/dycontent/images_upl/61...
62,63,Кефтедес - котлеты по-гречески,/recipes/recipe.php?rid=122724,"Греческие котлеты (вообще-то даже кипрские) ""к...","Продукты ,Фарш мясной - 500 г,Яйцо - 1 шт.,Хле...",//img1.russianfood.com/dycontent/images_upl/39...
63,64,Отбивные из свинины в яблочном кляре,/recipes/recipe.php?rid=167907,Замаринуйте свиные отбивные с натёртыми яблока...,"Продукты ,(на 3 порции),Свинина (мякоть) - 300...",//img1.russianfood.com/dycontent/images_upl/61...
